In [1]:
import os
from os.path import join as ospj
import time
from time import gmtime, strftime
import datetime
from munch import Munch
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from collections import deque

from network.model_zf import build_model
from core.checkpoint import CheckpointIO
from dataset.frame_dataset import FramesDataset, MotionDataset, DatasetRepeater
import network.utils as utils
import yaml
import random
from utils import Bar, Logger, AverageMeter, center
from tqdm import tqdm
from torch.nn.parallel.data_parallel import DataParallel
import pytorch_ssim
from torchsummary import summary
from torchvision import transforms
from tps.rand_tps import RandTPS
from network.vgg import VGG
import imp
from network.wing import FAN, HighPass
import copy

import warnings
warnings.filterwarnings("ignore")
import imageio

In [2]:
with open('config/train_transformer6.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
config = Munch(config)

In [3]:
# GPU Device
gpu_id = '0,1, 2'
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device " , use_cuda)

GPU device  True


In [4]:
torch.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)
np.random.seed(config.seed)

In [5]:
resume = False

In [6]:
class Solver(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.args.lr = float(self.args.lr)
        self.args.weight_decay = float(self.args.weight_decay)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.start = 10
        self.replay_memory = 10000
        self.replay_buffer = deque(maxlen=self.replay_memory)

        self.nets = build_model(args)
        # below setattrs are to make networks be children of Solver, e.g., for self.to(self.device)
        for name, module in self.nets.items():
            utils.print_network(module, name)
            setattr(self, name, module)

        if args.mode == 'train':
            self.optims = Munch()
            for net in self.nets.keys():
                self.optims[net] = torch.optim.Adam(params=self.nets[net].parameters(), lr=float(args.lr), betas=[args.beta1, args.beta2],
                                                   weight_decay=0)

            self.ckptios = [
                CheckpointIO(ospj(args.checkpoint_dir, '{:06d}_nets.ckpt'), **self.nets),
                CheckpointIO(ospj(args.checkpoint_dir, '{:06d}_optims.ckpt'), **self.optims)]
    

        self.to(self.device)
        for name, network in self.named_children():
            # Do not initialize the FAN parameters
            if ('ema' not in name) and ('fan' not in name):
                print('Initializing %s...' % name)
                network.apply(utils.he_init)
        
        # landmark
        self.fan = FAN(fname_pretrained=config.fname_fan)
        self.fan.to(self.device)
        self.fan.eval()
        self.hpf = HighPass(config.w_hpf, self.device)
        self.masking = transforms.RandomErasing(p=1.0, scale=(0.25, 0.33), ratio=(0.3, 0.33))
        self.clip_num = config.clip_num
        
        # eqv
        self.eqv_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ColorJitter(brightness=config.brightness, contrast=config.contrast, saturation=config.saturation, hue=config.hue),
            transforms.ToTensor(),])
        self.interp = nn.Upsample(size=(64, 64), mode='bilinear', align_corners=True)
        self.tps = RandTPS(width=config.img_size, height=config.img_size, batch_size=config.batch_size, sigma=config.sigma, 
                           border_padding=True, random_mirror=True, random_scale=(0.8, 1.1))
        self.tps.to(self.device)
        self.tps.eval()
        self.kl = nn.KLDivLoss().to(self.device)
        
        # perceptual loss
        self.vgg = VGG()
        MainModel = imp.load_source("MainModel", args.fname_ir)
        weight = torch.load(args.fname_vgg, map_location='cpu')
        self.vgg.load_state_dict(weight.state_dict(), strict=False)
        self.vgg.eval()
        self.vgg.to(self.device)
        

    def _save_checkpoint(self, step):
        for ckptio in self.ckptios:
            ckptio.save(step)

    def _load_checkpoint(self, step):
        for ckptio in self.ckptios:
            ckptio.load(step)

    def _reset_grad(self):
        for optim in self.optims.values():
            optim.zero_grad()
            
    @torch.no_grad()
    def evaluate(self, args, epoch, nets, loader):
        if not os.path.isdir(args.result_dir):
            os.makedirs(args.result_dir)
        result_target = os.path.join(args.result_dir, 'tar')
        result_gen = os.path.join(args.result_dir,'gen')
        if not os.path.isdir(result_target):
            os.makedirs(result_target)
        if not os.path.isdir(result_gen):
            os.makedirs(result_gen)
        
        bar = tqdm(total=len(loader), leave=False)
        ssim_meter, fid_meter = AverageMeter(), AverageMeter()
        for iteration, x in enumerate(loader):
            try:
                test_video = torch.tensor(np.concatenate(x['video'])) # (frame, c, w, h)
            except:
                continue
            num_frame = test_video.shape[0]
            k_frame = np.random.choice(num_frame-args.K, size=2, replace=False)
            source = test_video[[k_frame[0]]]
            target = test_video[[k_frame[1]]]
            real_land = self.fan.get_landmark(source.to(self.device))
            source_out = nets.transformer(real_land, self.fan.get_landmark(target.to(self.device)))
            source_out = self.get_clip(source_out)
            e_out = nets.embedder(source_out)
            source_gen = nets.generator(source.to(self.device)+F.interpolate(real_land, size=256, mode='bilinear'), e_out)
            ssim = float(pytorch_ssim.ssim(source_gen, target.to(self.device)))
            ssim_meter.update(ssim, iteration+1)
            
            # save for FID
            gen = source_gen.squeeze().cpu().detach().numpy()
            target = target.squeeze().cpu().detach().numpy()
            gen = gen.swapaxes(0, 1).swapaxes(1, 2)
            target = target.swapaxes(0, 1).swapaxes(1, 2)
            gen_img = Image.fromarray((gen*255).astype('uint8'))
            tar_img = Image.fromarray((target*255).astype('uint8'))
            gen_img.save(result_gen + '/{}.png'.format(iteration+1))
            tar_img.save(result_target + '/{}.png'.format(iteration+1))
            
            bar.set_description("Epoch:{:d}, SSIM: {:.8f}".format(epoch+1, ssim_meter.avg), refresh=True)
            bar.update()
        bar.close()
        val_logger.append([str(ssim_meter.avg)])
        return
    
    @torch.no_grad()
    def make_animation(self, args, nets, loaders):
        if not os.path.isdir(args.sample_dir):
            os.makedirs(args.sample_dir)
        K = 100
        random_list = np.random.choice(len(loaders.val.dataset), replace=False, size=2)
        source_image_idx = int(random_list[0])
        test_video_idx = int(random_list[1])
        train_video_idx = int(np.random.choice(len(loaders.src.dataset), size=1))
        # test animation
        source_image = loaders.val.dataset[source_image_idx]['video'][0]
        test_video = loaders.val.dataset[test_video_idx]['video'] # list [video](3, 256, 256)
        train_video = loaders.src.dataset[train_video_idx]['target'] # list [K](3, 256, 256)
        test_frame = len(test_video) if len(test_video) < K else K
        train_frame = len(train_video)
        predict_test, predict_train = [], []
        for i in range(test_frame):
            real_land = self.fan.get_landmark(source_image.to(self.device).unsqueeze(0))
            tar_land = self.fan.get_landmark(test_video[i].to(self.device).unsqueeze(0))
            out = nets.transformer(real_land, tar_land)
            out = self.get_clip(out)
            e_out = nets.embedder(out)
            gen = nets.generator(source_image.to(self.device).unsqueeze(0)+F.interpolate(real_land, size=256, mode='bilinear'), e_out, tar_land)
            predict_test.append(gen.cpu().detach().numpy().squeeze().swapaxes(0, 1).swapaxes(1,2))
        
        for i in range(train_frame):
            real_land = self.fan.get_landmark(source_image.to(self.device).unsqueeze(0))
            tar_land = self.fan.get_landmark(train_video[i].to(self.device).unsqueeze(0))
            out = nets.transformer(real_land, tar_land)
            out = self.get_clip(out)
            e_out = nets.embedder(out)
            gen  = nets.generator(source_image.to(self.device).unsqueeze(0)+F.interpolate(real_land, size=256, mode='bilinear'), e_out, tar_land)
            predict_train.append(gen.cpu().detach().numpy().squeeze().swapaxes(0,1).swapaxes(1,2))
        self.predict_test = predict_test
        source_image = (source_image*255).numpy().swapaxes(0,1).swapaxes(1,2).astype('uint8')
        source_img = Image.fromarray(source_image)
        source_img.save(config.sample_dir+ '/source.png')
        imageio.mimsave(os.path.join(config.sample_dir, 'test_gen.mp4'), [(frame*255).astype('uint8') for frame in predict_test], fps=24)
        imageio.mimsave(os.path.join(config.sample_dir, 'test_raw.mp4'), [(frame*255).numpy().astype('uint8').swapaxes(0,1).swapaxes(1,2) for frame in test_video], fps=24)
        imageio.mimsave(os.path.join(config.sample_dir, 'train_gen.mp4'), [(frame*255).astype('uint8') for frame in predict_train], fps=24)
        imageio.mimsave(os.path.join(config.sample_dir, 'train_raw.mp4'), [(frame*255).numpy().astype('uint8').swapaxes(0,1).swapaxes(1,2) for frame in train_video], fps=24)
        
    @torch.no_grad()
    def get_clip(self, out):
        out[out < self.clip_num] = 0.
        return out

    def train(self, loaders):
        args = self.args
        nets = self.nets
 
        for name in nets:
            nets[name] = DataParallel(nets[name])
            nets[name] = nets[name].to(self.device)
        optims = self.optims

        # resume training if necessary
        if args.resume_iter > 0:
            self._load_checkpoint(args.resume_iter)

        # batch
        for epoch in range(args.resume_iter, args.epochs):
            bar = tqdm(total=len(loaders.src)*args.K, leave=False)
            tf_loss = AverageMeter()
            wgan_loss, d_reg_loss = AverageMeter(), AverageMeter()
            g_latent_loss, g_cycle_loss, vgg_loss, fm_loss = AverageMeter(), AverageMeter(), AverageMeter(), AverageMeter()
            for i, inputs in enumerate(loaders.src):
                x_source, y_drive = inputs['source'], inputs['target']
                num_frame = len(y_drive)
                for f in range(num_frame):
                    self.replay_buffer.append((x_source, y_drive[f]))
                    
                    if len(self.replay_buffer) < self.start:
                        continue
                    minibatch = random.sample(self.replay_buffer, 1)
                    x_source_mb, y_drive_mb = minibatch[0][0], minibatch[0][1]
                    
                    train the transformer
                    x_out, landmark, tf_losses, tf_losses_latent = compute_tf_loss(nets, args, x_source_mb, y_drive_mb,
                                                                         transform=self.eqv_transform, tps=self.tps, interp=self.interp, 
                                                                         kl=self.kl, fan=self.fan, mask=self.masking,
                                                                         device=self.device)
                    self._reset_grad()
                    tf_losses.backward()
                    optims.transformer.step()
                    x_out = self.get_clip(x_out).detach()
                    
                    # train the discriminator
                    feat, d_loss, d_losses_latent = compute_d_loss(nets, args, x_source_mb, y_drive_mb, x_out, landmark[0], landmark[1], device=self.device)
                    self._reset_grad()
                    d_loss.backward()
                    optims.discriminator.step()
                    
                    # train the generator
                    g_loss, g_losses_latent = compute_g_loss(nets, args, x_source_mb, y_drive_mb, x_out, landmark[0], landmark[1], vgg=self.vgg, 
                                                             fan=self.fan, device=self.device)
                    self._reset_grad()
                    g_loss.backward()
                    optims.generator.step()
                    optims.embedder.step()
                    
                    wgan_loss.update(float(d_losses_latent.wgangp), x_source.size(0))
                    d_reg_loss.update(float(d_losses_latent.reg), x_source.size(0))
                    g_latent_loss.update(float(g_losses_latent.adv), x_source.size(0))
                    g_cycle_loss.update(float(g_losses_latent.cyc), x_source.size(0))
                    vgg_loss.update(float(g_losses_latent.vgg), x_source.size(0))
                    fm_loss.update(float(g_losses_latent.fm), x_source.size(0))
                    tf_loss.update(float(tf_losses_latent.tf), x_source.size(0))

                    bar.set_description("Ep:{:d}, D: {:.6f}, R1: {:.2f}, G: {:.6f}, Cyc: {:.6f}, Vgg: {:.6f}, FM: {:.6f}, TF: {:.6f}".format(
                                        epoch+1, wgan_loss.avg, d_reg_loss.avg, 
                                        g_latent_loss.avg, g_cycle_loss.avg, vgg_loss.avg, fm_loss.avg,
                                        tf_loss.avg), refresh=True)
                    bar.update()
            bar.close()

                # save model checkpoints
            logger.append([str(wgan_loss.avg)[:8], str(d_reg_loss.avg)[:8], 
                           str(g_latent_loss.avg)[:8], str(g_cycle_loss.avg)[:8], str(vgg_loss.avg)[:8], str(fm_loss.avg)[:8],
                           str(tf_loss.avg)[:8]])
            if (epoch+1) % config.save_every == 0:
                self._save_checkpoint(step=epoch+1)

            # compute SSIM and FID in test_set
            if (epoch+1) % config.eval_every == 0:
                self.evaluate(args, epoch, nets, loaders.val)
                
            self.make_animation(args, nets, loaders)
        
        self.evaluate(args, epoch, nets, loaders.val)

SyntaxError: invalid syntax (<ipython-input-6-63240b439f0b>, line 200)

In [ ]:
def compute_tf_loss(nets, args, x_real, y_org, transform, tps, interp, kl, fan, mask, device='cuda'):
    # TF_loss
    l1_loss = nn.L1Loss()
    x_real, y_org = x_real.to(device), y_org.to(device)
    x_landmark = fan.get_landmark(x_real)
    y_landmark = fan.get_landmark(y_org)
    y_land_masked = []
    for i in range(y_landmark.size(0)):
        y_land_masked.append(mask(y_landmark[i].clone()).cpu().numpy())
    y_land_masked = torch.from_numpy(np.array(y_land_masked))
    x_out = nets.transformer(x_landmark, y_land_masked)
    loss_tf = l1_loss(x_out, y_landmark)
    # Equivariance loss
    # real --> transform(color / tps) --> gen
#     softmax = nn.Softmax(dim=1)
#     images_cj = x_real
#     for b in range(images_cj.shape[0]):
#         images_cj[b] = transform(images_cj[b].cpu())
#     tps.reset_control_points()
#     images_tps = tps(images_cj.to(device))
#     images_tps_landmark = fan.get_landmark(images_tps)
#     pred_tps =  nets.transformer(images_tps_landmark, y_landmark)
#     # real --> gen --> transform(tps)
#     pred_d = x_out
#     pred_d = pred_d.detach()
#     pred_d.requires_grad = False
#     pred_tps_org = F.interpolate(torch.sum(tps(pred_d.repeat(1, 3, 1, 1)), 1, keepdim=True)/3., size=64)
#     # KL (first, second) / Center mse (first, second)
#     loss_eqv = kl(F.log_softmax(pred_tps, dim=1), F.softmax(pred_tps_org))
    loss = loss_tf * args.lambda_tf
    return x_out, (x_landmark, y_landmark), loss, Munch(tf=loss_tf.item())

def compute_d_loss(nets, args, x_real, y_org, x_out, x_landmark,  y_landmark, device='cuda'):
    # with real images
    x_real, y_org, x_out = x_real.to(device), y_org.to(device), x_out.to(device)
    x_real.requires_grad = True
    real_out, r_feat = nets.discriminator(x_real)
    loss_real = adv_loss(real_out, 1)
    # R1-reg
    loss_reg = r1_reg(real_out, x_real)
     # with fake images
    with torch.no_grad():
        e = nets.embedder(x_out)
        x_fake = nets.generator(x_real+F.interpolate(x_landmark, size=256, mode='bilinear'), e, y_landmark)
    fake_out, f_feat = nets.discriminator(x_fake)
    loss_fake = adv_loss(fake_out, 0)
    loss = loss_real + loss_fake + args.lambda_reg * loss_reg
    return (r_feat, f_feat),loss, Munch(wgangp=loss_real.item()+loss_fake.item(), reg=loss_reg.item())

def compute_g_loss(nets, args, x_real, y_org, x_out, x_landmark, y_landmark, vgg, fan, device='cuda'):
    # adversarial loss
    x_real, y_org, x_out= x_real.to(device), y_org.to(device), x_out.to(device)
    x_real_ld = x_real + F.interpolate(x_landmark, size=256, mode='bilinear')
    e = nets.embedder(x_out)
    x_fake = nets.generator(x_real_ld, e, y_landmark)
    out, _ = nets.discriminator(x_fake)
    # adv loss
    loss_adv = adv_loss(out, 1)
    
    # feature-matching loss
    l1_loss = nn.L1Loss()
    _, target_feat = nets.discriminator(y_org)
    loss_fm = l1_loss(e, target_feat.detach())
    
    # cycle-consistency loss
    x_fake_landmark = fan.get_landmark(x_fake)
    x_real_landmark = fan.get_landmark(x_real)
    x_fake_out = nets.transformer(x_fake_landmark, x_real_landmark)
    e_cyc = nets.embedder(x_fake_out)
    x_rec = nets.generator(x_fake+F.interpolate(x_fake_landmark, size=256, mode='bilinear'), e_cyc, x_landmark)
    loss_cyc = torch.mean(torch.abs(x_rec - x_real))
    
    # perceptual loss
    with torch.no_grad():
        vgg_x = vgg(y_org)
    with torch.autograd.enable_grad():
        vgg_xhat = vgg(x_fake)
        
    loss_vgg = 0
    for x_feat, xhat_feat in zip(vgg_x, vgg_xhat):
        loss_vgg += l1_loss(x_feat, xhat_feat)

    loss = loss_adv + args.lambda_cyc * loss_cyc + args.lambda_vgg * loss_vgg + args.lambda_fm * loss_fm
    return loss, Munch(adv=loss_adv.item(), cyc=loss_cyc.item(), vgg=loss_vgg.item(), fm=loss_fm.item())

def gradient_penalty(nets, real, fake, reg_lambda=10, device='cuda'):
    from torch.autograd import grad
    batch_size = real.shape[0]
    epsilon = torch.rand(batch_size, 1, 1, 1).to(device)
    merged = (epsilon * real) + ((1 - epsilon) * fake)
    # forward
    op = nets.discriminator(merged)
    
    # merted gradient
    gradient = grad(outputs=op, inputs=merged, create_graph=True, grad_outputs=torch.ones_like(op), 
                    retain_graph=True, only_inputs=True)[0]
    
    # calc penalty
    penalty = reg_lambda * ((gradient.norm(p=2, dim=1) - 1) ** 2).mean()
    return penalty
    
def moving_average(model, model_test, beta=0.999):
    for param, param_test in zip(model.parameters(), model_test.parameters()):
        param_test.data = torch.lerp(param.data, param_test.data, beta)

def adv_loss(logits, target):
    assert target in [1, 0]
    targets = torch.full_like(logits, fill_value=target)
    loss = F.binary_cross_entropy_with_logits(logits, targets)
    return loss

def r1_reg(d_out, x_in):
    # zero-centered gradient penalty for real images
    batch_size = x_in.size(0)
    grad_dout = torch.autograd.grad(
        outputs=d_out.sum(), inputs=x_in,
        create_graph=True, retain_graph=True, only_inputs=True
    )[0]
    grad_dout2 = grad_dout.pow(2)
    assert(grad_dout2.size() == x_in.size())
    reg = 0.5 * grad_dout2.view(batch_size, -1).sum(1).mean(0)
    return reg

In [ ]:
train_dataset = MotionDataset(config.root_dir, image_shape=config.frame_shape, id_sampling=True, is_train=True, random_seed=config.seed)
test_dataset = FramesDataset(config.root_dir, image_shape=config.frame_shape, id_sampling=True, is_train=False, random_seed=config.seed)

In [ ]:
train_dataset = DatasetRepeater(train_dataset, config.num_repeats)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, 
                              num_workers=config.num_workers, pin_memory=False, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=config.num_workers)

In [ ]:
loaders = Munch(src=train_loader, val=test_loader)

In [ ]:
solver = Solver(config)

In [ ]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    logger = Logger(os.path.join(config.checkpoint_dir, 'log.txt'), resume=True)
    val_logger = Logger(os.path.join(config.checkpoint_dir, 'val_log.txt'), resume=True)
else:
    logger = Logger(os.path.join(config.checkpoint_dir, 'log.txt'))
    val_logger = Logger(os.path.join(config.checkpoint_dir, 'val_log.txt'))
    logger.set_names(['WGAN-GP Loss', 'R1reg Loss', 'G-latent-adv Loss', 'G-cyclc Loss', 'Perceptual Loss', 'Feature-matching Loss', 'TF L1 Loss'])
    val_logger.set_names(['SSIM measure'])

In [ ]:
solver.train(loaders)